# Parabolic model problem
We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$
with a suitable advective field $b$ (the wind).

In [ ]:
import netgen.gui
%gui tk
import tkinter
from ngsolve import *
from netgen.geom2d import SplineGeometry

We generate geometry $(-1,1)^2$, mesh (Dirichlet boundaries everywhere):

In [ ]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u = fes.TrialFunction()
v = fes.TestFunction()

time = 0.0
dt = 0.001

We define the field $b$ (the wind) as $b(x,y) = (2y(1-x^2),-2x(1-y^2))$.

In [ ]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")

We define and assemble bilinear forms for convection-diffusion stiffness and mass matrix seperately.

Note that we choose a non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [ ]:
a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI (0.01*grad(u)*grad(v) + b*grad(u)*v)
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += SymbolicBFI (u*v)
m.Assemble()

We want to use a simple implicit Euler time stepping method, i.e.

$$
  (M + \Delta t A) u^{n+1} = M u^n + f^{n+1}
$$

or in an incremental form:

$$
  (M + \Delta t A) (u^{n+1} - u^n) = - \Delta t A u^n + f^{n+1}.
$$

The incremental form has the advantage that $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).

For the time stepping method we want to set up linear combinations of matrices.

(Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [ ]:
mstar = m.mat.CreateMatrix()
print(mstar)

To access the entries we use the vector of nonzero-entries:

In [ ]:
print(mstar.AsVector())

Using the vector we can build the linear combination of the a and the m matrix:

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [ ]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass    
print("")
time+=t_intermediate

## Supplementary: time-dependent r.h.s. data
Next, we want to consider time-dependent r.h.s. data $f=f(t)$:

To this end, we introduce a parameter t representing the time. 

A Parameter is a constant CoefficientFunction the value of which can be changed with the Set-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
Draw(gausspt,mesh,"ft")
time = 0.0
while time < 10 - 0.5 * dt:
    t.Set(time)
    Redraw(blocking=True)
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass        
    time += 1e-4

Accordingly we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += SymbolicLFI(gausspt*v)
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
Draw(gfu,mesh,"u")

In [ ]:
tstep = 20 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass    
print("")
time+=t_intermediate